## Introduction to Natural Language Processing
[**CC-BY-NC-SA**](https://creativecommons.org/licenses/by-nc-sa/4.0/deed.en)<br/>
Prof. Dr. Annemarie Friedrich / Fabio Mariani<br/>
Faculty of Applied Computer Science, University of Augsburg<br/>

#  Language Modeling

**Learning Goals**

* In this notebook, we will explore how to train and evaluate trigram (3-gram) language models using two sets of texts with different genres of the Brown Corpus.
* We will assess how well each model performs on a test set using *perplexity* as a metric.
* We will analyze the results to understand how genre, model parameters, and smoothing techniques influence language models performance.

## Resources

Before we build our language model, we need to import several modules and download necessary datasets:

- `nltk`: The Natural Language Toolkit, a popular library for working with NLP.
- `brown`, `gutenberg`: Two corpora provided by NLTK — the Brown Corpus contains categorized texts (e.g., fiction, news), while the Gutenberg Corpus includes classic literature.
- `math`: Used for mathematical operations, such as calculating logarithms and exponentials during perplexity evaluation.

We also use `nltk.download()` to ensure that the required corpora and tokenizers are available locally. This step downloads:
- `'gutenberg'` – Jane Austen and other literary texts.
- `'punkt_tab'` – Tokenizers.
- `'brown'` – A categorized corpus of American English.

In [ ]:
!pip install --user -U nltk

import nltk
from nltk.lm import Laplace, StupidBackoff
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.corpus import brown, gutenberg
from nltk.util import ngrams
from nltk import pad_sequence
import math
import pprint as pp

nltk.download('gutenberg')
nltk.download('punkt_tab')
nltk.download('brown')

## Preparing the Training and Test Data


We prepare the data used to train and test our trigram language models.

We extract sentences from the Brown Corpus, grouped by two distinct categories (genres):
- ``fiction``: used to train our first language model.
- ``news``: used to train our second language model.

&#9997; To what extent do we expect the two language models to differ?

_Your Answer:_

In [ ]:
# obtain sentences from Brown corpus object
model_fiction_sentences = brown.sents(categories='fiction')
model_news_sentences = brown.sents(categories='news')

# print the number of sentences that we use to train each language model
print(len(model_fiction_sentences))
print(len(model_news_sentences))

We use 200 sentences from "Emma" by Jane Austen (Gutenberg Corpus) as our test data, i.e., we will check how well the language models estimated on the training corpora match the distribution of n-grams in this new (unseen) text. The better a language model captures the token distributions and sequences in the target domain (or genre), the better the language model (for this domain).

&#9997; Do you expect the `fiction` or the `news` model to match this test text better?

_Your answer:_

In [ ]:
# We test on these sentences.
austen_sentences = gutenberg.sents('austen-emma.txt')[700:900]

## Training the N-gram Language Model

The following function trains an N-gram language model using the **Laplace smoothing** technique.

### Why Do We Need Smoothing in Language Models?

When training N-gram language models, we estimate the probability of a word given its preceding context (e.g., for trigrams, `P(w3 | w1, w2)`). However, we face a major challenge:

#### The Zero-Probability Problem

If an N-gram from the test set **never appeared** in the training data, the model assigns it a probability of **zero**.

- Even large training corpora can miss many valid n-gram combinations.
- This leads to entire sentences getting **zero probability**, which is not desired in practice - even if we have not seen a particular combination of words, the entire sentencen may still contain some likely parts and we would like to reflect this in our score.
- Computing perplexity would also turn complicated if our test set contains unseen sequences (which almost always happens in practice).

&#9997; Check the formular in the slides - why do zero probabilities provide a problem for computing perplexity?

_Your answer:_

**Smoothing** adjusts the probability distribution so that **no possible sequence gets a probability of zero**, even if it was unseen during training.

#### Laplace Smoothing

In this exercise, we use **Laplace smoothing**:

- It adds `1` to every possible N-gram count.
- This ensures **every possible word combination has a non-zero count**.
- It’s simple and effective for our purposes.

Smoothing is crucial for building robust language models that can handle new or rare word sequences.


In [ ]:
def train_ngram_model(corpus_sentences, n=3):
    train_data, vocab = padded_everygram_pipeline(n, corpus_sentences)
    model = Laplace(n)
    model.fit(train_data, vocab)
    return model

We can then train the 2 models:

In [ ]:
n = 3 # trigram

model_fiction = train_ngram_model(model_fiction_sentences, n)
model_news = train_ngram_model(model_news_sentences, n)

## Applying the Language Model on New Data

Once a language model is trained, we want to see how well it performs on new, unseen data. The function below takes a trained model and a list of tokenized sentences, then calculates the probability of each word based on its context using the model:

1) We add ``<s>`` padding to the beginning of the sentence so that the first n-grams can be formed correctly. For trigrams ``(n=3)``, this adds two ``<s>`` tokens: ``['<s>', '<s>', 'the', ...]``.

2) We split the sentence into overlapping sequences of tokens (n-grams). For trigrams: ``('<s>', '<s>', 'the'), ('<s>', 'the', 'dog'), ('the', 'dog', 'barked'), ... ``

3) Separate each n-gram into context and target word. 
    - n-gram: ``('the', 'dog', 'barked')``:
        - context (what the model see): ``('the', 'dog')``
        - target word (what we expect the model to predict) : ``barked``

4) We calculate the model probability to predict the expected word given the context.

5) The function returns a list of ``(word, probability)`` for all the words in new data.


In [ ]:
def apply_model(model, tokenized_sentences, n):
    prob_t = []
    for sentence in tokenized_sentences:
        padded_tokens = pad_sequence(sentence, n=n, pad_left=True, left_pad_symbol='<s>')
        for ngram in ngrams(padded_tokens, n):
            context = ngram[:-1]
            word = ngram[-1]
            prob = model.score(word, context)
            prob_t.append((word, prob, context))
    return prob_t

In [ ]:
model_fiction_probs = apply_model(model_fiction, austen_sentences, n)
model_news_probs = apply_model(model_news, austen_sentences, n)

# Take a look at the data structure
pp.pprint(model_fiction_probs[:10])

## Exercise: Calculate Perplexity

Now that we’ve evaluated word probabilities using our language models, it’s time to **quantify** how surprised the models are with new data (test text) using **perplexity**.

---

### What is Perplexity?

Perplexity measures how well a language model predicts a sequence of words. It can be interpreted as:

> "**How surprised** is the model when it sees the actual test data?"

- A **lower** perplexity score means the model is more confident and better at predicting the text.
- A **higher** perplexity indicates more uncertainty or poor prediction.

Mathematically, for a sequence of N words with probabilities \( p_1, p_2, ..., p_N \), perplexity is defined as:

$$
\text{Perplexity} = \exp \left( -\frac{1}{N} \sum_{i=1}^{N} \log p_i \right)
$$

Where:
- \( pi \) is the predicted probability of the i-th word (following a history whose length depends on the language model),
- \( N \) is the total number of predicted words.


### Your Task

Complete the following function to compute perplexity based on the list of predicted probabilities.

**Input**

``words_probs``: A list of tuples like ``(word, probability, context)``

Example: ``[('the', 0.003, ('<s>', '<s>')), ('dog', 0.02, ('<s>', 'the'))]``


**Output**

Returns a single float value: the computed perplexity of the model over the test data.



The function may use Python's built-in ``math`` module:
- `math.log(prob)` calculates the natural logarithm (ln) of a probability.
- `math.exp(x)` computes the exponential \( e^x \).
You can read more here: [math documentation](https://docs.python.org/3/library/math.html)

In [ ]:
def calculate_perplexity(words_probs):
    # your code here
    pass

In [ ]:
ppl_fiction = calculate_perplexity(model_fiction_probs)
ppl_news = calculate_perplexity(model_news_probs)


print(f'Perplexity for model_fiction: {ppl_fiction}')
print(f'Perplexity for model_news: {ppl_news}')

 &#9997; What do you observe? What does this mean? Do the observation match your expectations?

 _ Your answer:_

  &#9997; Experiment with at least two parameters, e.g., change the length of the history ($n$) or experiment with a different smoothing method, adding training data (check out how to use text from several categories of the Brown corpus, for example), etc. and briefly describe your findings. Submit the version of the notebook containing the answers and code for the original question, but keep your variants either below or comment them out.